In [1]:
import pymssql
import pandas as pd
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('db_config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_name = config['database']['dbname']

sql_conn = pymssql.connect(db_host, db_user, db_password, db_name)

In [3]:
def get_data(query, columns = None):
    """Query in SQL STRING
    Columns = List of strings representing the columns for the df"""
    mycursor = sql_conn.cursor()
    mycursor.execute(query)
    result = mycursor.fetchall()
    data_list = list(result)
    if columns == None:
        df = pd.DataFrame(data_list)
        return df
    else:
        df = pd.DataFrame(data_list, columns= columns)
        return df

In [7]:
#Find all events where opioid dependence was diagnosed (has to appear 2+ times)

query_diagnosis_columns = """SELECT *
FROM syscolumns
WHERE id=OBJECT_ID('CDW_NEW.deid_uf.DiagnosisEventFact') """
diagnosis_columns = get_data(query_diagnosis_columns)[0].to_list()
query_opioid = """
    SELECT * 
    FROM CDW_NEW.deid_uf.DiagnosisEventFact
    WHERE patientdurablekey IN (
        SELECT patientdurablekey
        FROM CDW_NEW.deid_uf.DiagnosisEventFact
        WHERE DiagnosisName LIKE '%opioid%'
        AND DiagnosisName LIKE '%dependence%'
        AND DiagnosisName NOT LIKE '%excluding opioid%'
        GROUP BY patientdurablekey
        HAVING COUNT(*) >= 2
)

"""
# query_ms= """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
# WHERE DiagnosisName LIKE '%multiple sclerosis%' """
opioid_diagnosis = get_data(query_opioid, diagnosis_columns)
opioid_diagnosis

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f058a72a910>>
Traceback (most recent call last):
  File "/pool0/home/sanjay.r/anaconda3/envs/sergio_rotation/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


,DeidLds,PatientDurableKey,DiagnosisEventKey,DiagnosisKey,DiagnosisName,PatientKey,AgeKey,AgeKeyValue,EncounterKey,DepartmentKey,...,DocumentedByProviderType,DocumentedByProviderPrimarySpecialty,DocumentedByProviderDurableKey,Type,Status,PresentOnAdmission,HospitalDiagnosis,EmergencyDepartmentDiagnosis,Chronic,Count
0,deid_uf,D0B224066983D9,D7782CDD6EA167,1404806,Urinary tract infection in mother during first...,DB07CF608A037E,7441004.0,NaN,D10E7C3DBC0082,3614,...,*Unknown,*Unknown,400455,Encounter Diagnosis,Active,*Not Applicable,NaN,NaN,0.0,1
1,deid_uf,DD9BDF885B89C1,DA5690AEA90C36,412710,Examination of participant in clinical trial,DDB06FA731C5A1,28710004.0,NaN,D3007A259A4D61,3614,...,*Unknown,*Unknown,400455,Encounter Diagnosis,Active,*Not Applicable,NaN,NaN,0.0,1
2,deid_uf,DD9BDF885B89C1,DA931F638751ED,1603965,Metastasis from hormone-refractory prostate ca...,DDB06FA731C5A1,28710004.0,NaN,D3007A259A4D61,3614,...,*Unknown,*Unknown,400455,Encounter Diagnosis,Active,*Not Applicable,NaN,NaN,0.0,1
3,deid_uf,DD9BDF885B89C1,D6F41ACA93ADD7,506181,Prostate cancer (CMS code),DDB06FA731C5A1,28710004.0,NaN,D3007A259A4D61,3614,...,*Unknown,*Unknown,400455,Encounter Diagnosis,Active,*Not Applicable,NaN,NaN,0.0,1
4,deid_uf,DD83451550CD83,D517A2B4759284,1313193,"Type 2 diabetes mellitus without complication,...",DA84AAC1B056BC,13350004.0,NaN,DB1A845B82CE5A,3614,...,*Unknown,*Unknown,400455,Encounter Diagnosis,Active,*Not Applicable,NaN,NaN,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4787480,deid_uf,D9E7F3E5ED8780,D982EF5D631C4D,386307,Routine general medical examination at a healt...,D2CBDF48F77BCB,17489004.0,NaN,D1F06D2AD72749,-1,...,*Unspecified,*Unspecified,-1,STOR Med Rec Abstracted Dx,Unknown,,NaN,NaN,NaN,1
4787481,deid_uf,D3E24C8114EFF8,D6AF37442AA2C2,386307,Routine general medical examination at a healt...,DE6D132E61DDB1,12395004.0,NaN,D42B304F6370F0,-1,...,*Unspecified,*Unspecified,-1,STOR Med Rec Abstracted Dx,Unknown,,NaN,NaN,NaN,1
4787482,deid_uf,D99B370ED56CA4,DDF379AEBF744C,386307,Routine general medical examination at a healt...,D03C995CE305B1,15752004.0,NaN,D06B8C27AAC98C,-1,...,*Unspecified,*Unspecified,-1,STOR Med Rec Abstracted Dx,Unknown,,NaN,NaN,NaN,1
4787483,deid_uf,D99B370ED56CA4,D13A09BEDAFB8E,386307,Routine general medical examination at a healt...,D03C995CE305B1,15918004.0,NaN,D55224C25A3BF0,-1,...,*Unspecified,*Unspecified,-1,STOR Med Rec Abstracted Dx,Unknown,,NaN,NaN,NaN,1


In [5]:
type(opioid_diagnosis["PatientDurableKey"])

pandas.core.series.Series

In [6]:
#Find all events where opioids were prescribed to patients— limited to oxycontin and oxycodone

# dependent_record_keys = opioid_diagnosis["PatientDurableKey"].tolist()
# dependent_record_keys_str = ', '.join([str(key) for key in dependent_record_keys])

query_med_columns = """SELECT *
FROM syscolumns
WHERE id=OBJECT_ID('CDW_NEW.deid_uf.MedicationOrderFact') """
med_columns = get_data(query_med_columns)[0].to_list()
query_opioid_presc = f"""
SELECT * 
FROM CDW_NEW.deid_uf.MedicationOrderFact
WHERE (MedicationName LIKE '%oxycontin%'
OR MedicationName LIKE '%oxycodone%')
"""
opioid_prescribed = get_data(query_opioid_presc, med_columns)
opioid_prescribed

,DeidLds,PatientDurableKey,MedicationOrderKey,PatientKey,EncounterKey,DepartmentKey,BedName,DepartmentName,DepartmentSpecialty,PharmacyKey,...,ReorderedFromPrescription,Verbal,RequiresCosign,Cosigned,VerbalSigned,Protocol,HighPriority,Prn,Count,SupervisedByProviderDurableKey
0,deid_uf,D7FB40E1815E55,DC8A7CFC1C4FC6,D85E6E3E0299D2,DC1398D1099B3A,17425,*Not Applicable,RAD IR PCMB 2,Interventional Radiology,-2,...,0.0,0.0,1.0,NaN,1.0,0.0,0.0,1.0,1,-1
1,deid_uf,D734CCE8B11FFC,D6D15AB0D88A00,D99181B2EBAFCF,DBF0BB48382BA9,22,*Not Applicable,12L MUSCULOSKELETAL,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
2,deid_uf,D734CCE8B11FFC,D83908A863B4EC,D99181B2EBAFCF,DBF0BB48382BA9,22,*Not Applicable,12L MUSCULOSKELETAL,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
3,deid_uf,DAE1A2E2EFC7C3,DA146254AF5871,DA57C0AD4CD58B,DFEF081572F155,240,*Not Applicable,TRANSFUSION PARN,Infusion and Transfusion,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
4,deid_uf,DAF6243BFEC85B,D40E920D3C1E47,DCE0E28819F66F,D25C75C4935D5D,43,*Not Applicable,EMERGENCY DEPT PARN,Emergency Medicine,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313980,deid_uf,DD4084D18EB42B,D60F15017DB99A,D16A42CFF5FCAF,DC1E0199185725,71,*Not Applicable,HEMODIALYSIS PARN,Dialysis,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313981,deid_uf,DB01F973DF8826,D0EB1767C6D3C4,DA3732C158C161,D9F00EDE7978C9,28,*Not Applicable,L14 MEDSURG-ONC/BMT B,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313982,deid_uf,DD3E1D2D7E07B0,D1B87DC215E3AB,D936C571E2C111,D23CA6867F15A2,240,*Not Applicable,TRANSFUSION PARN,Infusion and Transfusion,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313983,deid_uf,D23027576D834F,DB56F389BFC07D,D25319CB99B0E9,D8B1DEA0D6782E,2545,*Not Applicable,APHERESIS PARN,Hematology and Oncology,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1


In [7]:
#Merge the prescribed and diagnosis dataframes
opioid_composite = pd.merge(opioid_diagnosis, opioid_prescribed, on="PatientDurableKey", suffixes=('_diagnosis', '_prescribed'))

# Filter where the startDateKey from opioid_diagnosis is smaller than the startDateKey from opioid_prescribed
filtered_opioid_composite = opioid_composite[(opioid_composite['StartDateKey_diagnosis'] < opioid_composite['StartDateKey_prescribed']) & (opioid_composite['StartDateKey_diagnosis'] > 0)]

In [8]:
filtered_opioid_composite[["DiagnosisName", "StartDateKey_diagnosis", "StartDateKey_prescribed"]]

,DiagnosisName,StartDateKey_diagnosis,StartDateKey_prescribed
397,Opioid dependence (CMS code),20000411,20201229
398,Opioid dependence (CMS code),20000411,20201229
399,Opioid dependence (CMS code),20000411,20201229
400,"Opioid use disorder, severe, dependence (CMS c...",20200529,20200929
401,"Opioid use disorder, severe, dependence (CMS c...",20200529,20230414
...,...,...,...
1012703,Opioid dependence (CMS code),20161204,20161218
1012704,Opioid dependence (CMS code),20161204,20230301
1012705,Opioid dependence (CMS code),20161204,20220202
1012706,Opioid dependence (CMS code),20161204,20221210


In [9]:
# How many unique patients have been prescribed opioids AFTER being diagnosed with opioid dependency?
filtered_opioid_composite['PatientDurableKey'].nunique()

3399

## Doing the same thing with Alcohol Dependence (Alc Dependence --> Opioid Prescription)

In [10]:
#Find all events where alcohol dependence was diagnosed

query_alcohol = """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
WHERE DiagnosisName LIKE '%alcohol dependence%'
"""
# query_ms= """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
# WHERE DiagnosisName LIKE '%multiple sclerosis%' """
alcohol_diagnosis = get_data(query_alcohol, diagnosis_columns)
alcohol_diagnosis

,DeidLds,PatientDurableKey,DiagnosisEventKey,DiagnosisKey,DiagnosisName,PatientKey,AgeKey,AgeKeyValue,EncounterKey,DepartmentKey,...,DocumentedByProviderType,DocumentedByProviderPrimarySpecialty,DocumentedByProviderDurableKey,Type,Status,PresentOnAdmission,HospitalDiagnosis,EmergencyDepartmentDiagnosis,Chronic,Count
0,deid_uf,D6EF9DC5B3421D,D7DE0D9A0013E7,491111,"Alcohol dependence, uncomplicated (CMS code)",D092447008A4BB,NaN,NaN,D6CE25DCB78153,23898,...,Social Worker,Psychology,238191,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
1,deid_uf,D346A970BA68A9,D6E092DC8F57DF,325545,"Alcohol dependence, in remission (CMS code)",D0152A95E3FF20,NaN,NaN,D9D2259AC74EDC,1010,...,Nurse Practitioner,General Surgery,117182,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
2,deid_uf,D6EF9DC5B3421D,DF953FD3665646,491111,"Alcohol dependence, uncomplicated (CMS code)",D092447008A4BB,NaN,NaN,DA9E237677BBDB,23898,...,Social Worker,Psychology,238191,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
3,deid_uf,DDC5B24932110A,DE4E777B2CA849,728333,"Other and unspecified alcohol dependence, in r...",DB82060E7453E4,26312004.0,NaN,D7B381D2CB6211,-1,...,*Unspecified,*Unspecified,-1,STOR Visit Dx,Unknown,,NaN,NaN,NaN,1
4,deid_uf,D045C8D125FFEC,DE2FC743D526E0,298581,"Other and unspecified alcohol dependence, unsp...",DDB054821718E8,13337004.0,NaN,DA0E922B7F3E53,-1,...,*Unspecified,*Unspecified,-1,STOR Visit Dx,Unknown,,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108006,deid_uf,D42E698F52C436,DF8C78A3022689,491111,"Alcohol dependence, uncomplicated (CMS code)",D95566DFB28B77,NaN,NaN,D551D2BC3E9629,-2,...,*Not Applicable,*Not Applicable,-2,Billing Diagnosis,Active,*Unspecified,NaN,NaN,NaN,1
108007,deid_uf,D425F90A5637FD,D8B15049E5E6CE,298581,"Other and unspecified alcohol dependence, unsp...",D0590C94A749E9,21436004.0,NaN,DE4D5F39DCF17C,44,...,Physician,Radiology,184767,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
108008,deid_uf,D84F65A38DC0E7,D5699E72760E7C,491111,"Alcohol dependence, uncomplicated (CMS code)",DE70C186DC34CA,23174004.0,NaN,DB5B5EAB61509E,68,...,Physician,Cardiology,228484,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
108009,deid_uf,D8EC9E5B4A6FBB,D6450D9F78BE04,491111,"Alcohol dependence, uncomplicated (CMS code)",D2909E817B22E8,NaN,NaN,DE18699D2D7263,-2,...,*Not Applicable,*Not Applicable,-2,Outgoing Claim Diagnosis,Active,No,1.0,0.0,NaN,1


In [11]:
#Merge the prescribed and diagnosis dataframes
alcohol_composite = pd.merge(alcohol_diagnosis, opioid_prescribed, on="PatientDurableKey", suffixes=('_diagnosis', '_prescribed'))

# Filter where the startDateKey from opioid_diagnosis is smaller than the startDateKey from opioid_prescribed
filtered_alcohol_composite = alcohol_composite[(alcohol_composite['StartDateKey_diagnosis'] < alcohol_composite['StartDateKey_prescribed']) & (alcohol_composite['StartDateKey_diagnosis'] > 0)]

In [12]:
filtered_alcohol_composite[["DiagnosisName", "StartDateKey_diagnosis", "StartDateKey_prescribed"]]

,DiagnosisName,StartDateKey_diagnosis,StartDateKey_prescribed
62,Alcohol dependence with unspecified alcohol-in...,20210208,20210601
72,"Alcohol dependence, uncomplicated (CMS code)",20171031,20180320
73,"Alcohol dependence, uncomplicated (CMS code)",20171031,20220615
74,"Alcohol dependence, uncomplicated (CMS code)",20171031,20220617
82,"Alcohol dependence with intoxication, uncompli...",20190603,20230913
...,...,...,...
670728,"Alcohol dependence, uncomplicated (CMS code)",20160410,20160414
670729,"Alcohol dependence, uncomplicated (CMS code)",20160410,20160412
670730,"Alcohol dependence, uncomplicated (CMS code)",20160410,20160412
670731,"Alcohol dependence, uncomplicated (CMS code)",20160410,20160413


In [14]:
# How many unique patients have been prescribed opioids AFTER being diagnosed with alcohol dependency?
filtered_alcohol_composite['PatientDurableKey'].nunique()

3721

# How many prescribed opioids with NO previous dependence

In [15]:
#Find all events where no dependence was diagnosed (Is this the right way to do it? Seems off, takes wayy too long to run...)

query_control = """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
WHERE DiagnosisName NOT LIKE '%dependence%'
"""
# query_ms= """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
# WHERE DiagnosisName LIKE '%multiple sclerosis%' """
control_diagnosis = get_data(query_control, diagnosis_columns)
control_diagnosis

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f741c148910>>
Traceback (most recent call last):
  File "/pool0/home/sanjay.r/anaconda3/envs/sergio_rotation/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
